In [2]:
def get_xyxy_from_txt(label):
    gt_boxes = list(map(float, label.split(' ')[1:]))
    x_center = gt_boxes[0]*1024
    y_center = gt_boxes[1]*128
    box_width = gt_boxes[2]*1024
    box_height = gt_boxes[3]*128
    
    x1 = int(x_center - (box_width / 2))
    y1 = int(y_center - (box_height / 2))
    x2 = int(x_center + (box_width / 2))
    y2 = int(y_center + (box_height / 2))

    return [x1, y1, x2, y2]

In [3]:
def calculate_iou(box1, box2):
    x1_box1, y1_box1, x2_box1, y2_box1 = box1
    x1_box2, y1_box2, x2_box2, y2_box2 = box2

    x_left = max(x1_box1, x1_box2)
    y_top = max(y1_box1, y1_box2)
    x_right = min(x2_box1, x2_box2)
    y_bottom = min(y2_box1, y2_box2)
    
    intersection_area = max(0, x_right - x_left) * max(0, y_bottom - y_top)
    
    area_box1 = (x2_box1 - x1_box1) * (y2_box1 - y1_box1)
    area_box2 = (x2_box2 - x1_box2) * (y2_box2 - y1_box2)
    
    union_area = area_box1 + area_box2 - intersection_area
    
    iou = intersection_area / union_area if union_area > 0 else 0
    return iou

def associate_boxes(boxes1, boxes2):
    associations = []
    for box1 in boxes1:
        max_iou = 0
        associated_box = None
        for box2 in boxes2:
            iou = calculate_iou(box1, box2)
            if iou > max_iou:
                max_iou = iou
                associated_box = box2
        associations.append((box1, associated_box, max_iou))
    return associations


# list1 = [(10, 20, 50, 60), (30, 40, 70, 80)]
# list2 = [(12, 22, 52, 62), (25, 35, 65, 75), (40, 50, 90, 100)]

# associations = associate_boxes(list1, list2)

# # Display associations (box from list1, associated box from list2, IoU)
# for box1, box2, iou in associations:
#     print(f"Box1: {box1}, Associated Box2: {box2}, IoU: {iou}")

In [4]:
import os
import cv2
import math
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from natsort import natsorted
from ouster import client
from ouster import pcap
from ouster.client import Scans, XYZLut, SensorInfo, destagger
from ouster.client.data import LidarScan
from ultralytics import YOLO
from contextlib import closing
from collections import defaultdict


output_dict = {3:["GO AHEAD", (0, 255, 0)], 2:["BE CAREFUL", (0, 155, 100)], 1:["SLOW DOWN", (0, 100, 155)], 
                0:["BREAK", (0, 0, 255)]}

# Load the YOLOv8 model
model = YOLO('../weights/best_3000_s_100.pt')

# Paths to pcap and json files
metadata_path = "C:/Users/szyme/Ouster/data/PKR_test1/test4.json"
pcap_path = "C:/Users/szyme/Ouster/data/PKR_test1/test4.pcap"

# Making PacketSource from data
with open(metadata_path, 'r') as f:
    metadata = client.SensorInfo(f.read())

fps = int(str(metadata.mode)[-2:])
width = int(str(metadata.mode)[:4])
height = int(str(metadata.prod_line)[5:])

pcap_file = pcap.Pcap(pcap_path, metadata)

xyz_lut = client.XYZLut(metadata) #call cartesian lookup table

# Store the track history
track_history = defaultdict(lambda: [])     # dictionary: {key=id, value=[xyz_val_0, ... ,xyz_val_99]}
box_history = defaultdict(lambda: [])
iou_values = []

directory_path = 'groundtruthing_labels/'

with closing(Scans(pcap_file)) as scans:

    save_path = "C:/Users/szyme/Ouster/Dangerous-situations-with-pedastrians/results_mp4"
    vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    files = os.listdir(directory_path)
    files_sorted = natsorted(files)

    i = 0
    for scan, filename in zip(scans, files_sorted):
        i += 1

        f = os.path.join(directory_path, filename)
        if os.path.isfile(f):
            labels_file = f
        with open(labels_file, 'r') as file:
            labels = file.readlines()

        sig_field = scan.field(client.ChanField.SIGNAL)
        sig_destaggered = destagger(metadata, sig_field)
        scaling_factor = 0.004
        scaled_arr = sig_destaggered / (0.5 + scaling_factor * sig_destaggered)
        signal_image = scaled_arr.astype(np.uint8)
        combined_img = np.dstack((signal_image, signal_image, signal_image))

        xyz_destaggered = client.destagger(metadata, xyz_lut(scan))

        results = model.track(source=combined_img, persist=True, imgsz=1024, tracker='../trackers/tracker.yaml', verbose=False)

        boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)

        if (results[0].boxes.id == None):
            ids = ''
        else:
            ids = results[0].boxes.id.cpu().numpy().astype(int)

        priority = 3
        distance = 0

        gt_boxes = []
        for label in labels:
            gt_boxes.append(get_xyxy_from_txt(label))

        associations = associate_boxes(gt_boxes, boxes)
        for box1, box2, iou in associations:
            print(f"Box1: {box1}, Associated Box2: {box2}, IoU: {iou}")
            iou_values.append(iou)

        for box, id, label in zip(boxes, ids, labels):
            # x1, y1, x2, y2 = get_xyxy_from_txt(label)
            # print(f"{x1}, {y1}, {x2}, {y2}")

            # print(box)
            center_x, center_y = int((box[0] + box[2])/2), int((box[1] + box[3])/2) #

            xyz_val = xyz_destaggered[(center_y, center_x)] #get the (x,y,z) coordinates with the lookup table 

            track = track_history[id] #save the (x,y,z) coordinates for distance calculation
            track.append(xyz_val)

            new_box = box_history[id]
            new_box.append((box[0], box[1], box[2], box[3]))

            cv2.rectangle(combined_img, (box[0], box[1]), (box[2], box[3]), output_dict[priority][1], 2)
            cv2.rectangle(combined_img, (box[0], box[1]+2), (box[0]+20+len(str(id))*11, box[1]-12), (255, 255, 255), -1)
            cv2.putText(combined_img, f"Id {id}", (box[0], box[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        cv2.putText(combined_img, f"{output_dict[priority][0]}", (470, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, output_dict[priority][1], 2)

        if True:
            cv2.imshow("YOLOv8 Tracking", combined_img)
            cv2.waitKey(1)  # 1 millisecond

        vid_writer.write(combined_img)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        if i >= 1000:
            break

    vid_writer.release()
    cv2.destroyAllWindows()

NameError: name 'directory' is not defined